In [23]:
import sys
import os
sys.path.append(os.path.abspath("src"))

import accessibility

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import os

CITY_NAME = "Munich"
STOPS_CSV = "data/stops.txt"
DISTRICTS_GEOJSON = "data/munich_districts.geojson"
OUTPUT_DIR = "output"

# Buffer distances in meters
DISTANCES = [300, 500, 800]

# Colormap configuration
COLOR_MAP_NAME = "Bmpoop"  # fallback to 'viridis' if not found
TICKS = [0, 20, 40, 60, 80, 100]
VMIN, VMAX = 0, 100

### LOAD AND PROCESS DATA

In [24]:
# Load stops and drop missing coordinates

def load_and_process_stops(stops_csv, distances):
    stops = pd.read_csv(stops_csv)
    stops = stops.dropna(subset=['stop_lat', 'stop_lon'])
    
    # Создаем GeoDataFrame с точками (координатами) для остановок
    stops_gdf = gpd.GeoDataFrame(
        stops,
        geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
        crs='EPSG:4326'
    )
    print(f"[INFO] Loaded {len(stops)} stops")

    # Проекцируем в систему координат для метрик (в метрах)
    stops_gdf = stops_gdf.to_crs(epsg=3857)

    # Генерируем буферы для каждой дистанции
    for dist in distances:
        stops_gdf[f'buffer_{dist}'] = stops_gdf.geometry.buffer(dist)

    # Создаем union для каждого буфера
    buffer_unions = {
        dist: stops_gdf[f'buffer_{dist}'].union_all()
        for dist in distances
    }
    
    return stops_gdf, buffer_unions

### MAP CHARACTERISTIC SETTINGS

In [1]:
from pypalettes import load_cmap

def setup_visualization(color_map_name, output_dir):
    try:
        cmap = load_cmap(color_map_name, cmap_type='continuous')
    except Exception:
        print("[WARNING] Failed to load custom colormap. Using 'viridis'.")
        cmap = plt.get_cmap('viridis')

    try:
        plt.rcParams['font.family'] = 'Helvetica Neue'
    except RuntimeError:
        plt.rcParams['font.family'] = 'Arial'

    os.makedirs(output_dir, exist_ok=True)
    
    return cmap

### INDIVIDUAL MAPS

In [26]:
def generate_individual_maps(districts, distances, cmap, output_dir, city_name, vmin, vmax, ticks):
    for dist in distances:
        fig, ax = plt.subplots(figsize=(10, 8))
        column = f'coverage_{dist}_pct'

        districts.plot(
            column=column,
            cmap=cmap,
            linewidth=0.5,
            ax=ax,
            edgecolor='0.8',
            legend=True,
            vmin=vmin,
            vmax=vmax,
            legend_kwds={
                'orientation': "vertical",
                'shrink': 0.7,
                'ticks': ticks
            }
        )

        ax.set_title(f"Public Transport Stop Coverage by Walking Distance in {city_name} (% of District Area)", fontsize=14)
        ax.set_axis_off()

        filename = os.path.join(output_dir, f"{city_name.lower().replace(' ', '_')}_coverage_{dist}m.png")
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"[INFO] Saved individual map: {filename}")

### COMBINED MAP

In [27]:
def generate_combined_map(districts, distances, cmap, output_dir, city_name, vmin, vmax, ticks):
    fig, axes = plt.subplots(1, 3, figsize=(22, 8), constrained_layout=False)

    for ax, dist in zip(axes, distances):
        column = f'coverage_{dist}_pct'
        districts.plot(
            column=column,
            cmap=cmap,
            linewidth=0.5,
            ax=ax,
            edgecolor='0.8',
            vmin=vmin,
            vmax=vmax
        )
        ax.set_title(f"{dist}m Coverage", fontsize=14)
        ax.set_axis_off()

    # Add colorbar below all maps
    cbar_ax = fig.add_axes([0.3, 0.08, 0.4, 0.02])
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal', ticks=ticks)
    cbar.set_label("Coverage (%)", fontsize=12)
    cbar.ax.tick_params(labelsize=10)

    fig.suptitle(f"Public Transport Stop Coverage by Walking Distance in {city_name} (% of District Area)", fontsize=18, y=0.95)

    combined_path = os.path.join(output_dir, f"{city_name.lower().replace(' ', '_')}_coverage_combined.png")
    plt.savefig(combined_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] Saved combined map: {combined_path}")

In [28]:
# 1. Загрузим и обработаем остановки
stops_gdf, buffer_unions = load_and_process_stops(STOPS_CSV, DISTANCES)

# 2. Настроим визуализацию
cmap = setup_visualization(COLOR_MAP_NAME, OUTPUT_DIR)

# 3. Сгенерируем индивидуальные карты для каждой доступности
generate_individual_maps(districts, DISTANCES, cmap, OUTPUT_DIR, CITY_NAME, VMIN, VMAX, TICKS)

# 4. Сгенерируем комбинированную карту
generate_combined_map(districts, DISTANCES, cmap, OUTPUT_DIR, CITY_NAME, VMIN, VMAX, TICKS)

[INFO] Loaded 10926 stops
[INFO] Saved individual map: output/munich_coverage_300m.png
[INFO] Saved individual map: output/munich_coverage_500m.png
[INFO] Saved individual map: output/munich_coverage_800m.png
[INFO] Saved combined map: output/munich_coverage_combined.png
